### **Proyecto 2 - Ingesta, Transformación, Carga y Reportería**

#### **Arquitectura del Proyecto**

<center><img src="https://i.postimg.cc/t4j2B2gs/adf455.png"></center>
<center><img src="https://i.postimg.cc/rpygksxc/adf456.png"></center>

### **Paso 1 - Data Ingestion**
_____

1. Crear un Self-hosted Integrated runtime en ADF

2. Descargar y configurar el Self-hosted IR en nuestro entorno local

3. Pasos a seguir en nuestra Ingesta de datos
    *   Crear un contenedor en ADLS donde almacenar la data
    *   Crear un Pipeline de ingesta de datos
    *   Crear Linked Services de origen y destino
    *   Crear datasets de origen y destino

<center><img src="https://i.postimg.cc/MZgJXZMw/adf458.png"></center>

##### **Paso 1.1 - Azure Key Vault**

1. Crear el recurso de Azure Key Vault

2. Crear directivas de acceso para Azure Data Factory (Access Policies) en Azure Key Vault

3. Crear secretos de usuario y contraseña para utilizarlo en la conexión del Linked Service con nuestro entorno local

4. Crear secreto para la Access Key (Account Key) de nuestra cuenta de almacenamiento ADLS

##### **Paso 1.2 - Creación de Linked Services de origen y destino**

1. Crear el Linked Service **LS_Keyvault** que hace referencia a Azure Key Vault

2. Crear el Linked Service de origen **LS_Onprem_File** que hace referencia al almacenamiento de nuestro local

3. Crear el Linked Service de destino **LS_adls_ingest** que hace referencia a nuestro ADLS

##### **Paso 1.3 - Creación de Datasets de origen y destino**

1. Crear el Dataset de origen **DS_Onprem_File** que hace referencia al almacenamiento de nuestro local

2. Crear el Dataset de destino **DS_adls_ingest** que hace referencia a nuestro ADLS, a la ruta **raw/ingest**. 
    Siendo **raw** el contenedor y **ingest** el directorio

##### **Paso 1.4 - Creación de un Pipeline**

1. Crear el Pipeline de ingesta de datos **PL_Onprem_ADLS_Ingest**, el cual contendrá una actividad **Copy data**

2. Crearemos un Pipeline más eficiente realizando una **Carga Incremental (Incremental load)** basandonos en la última fecha    
   modificada. Para ello, necesitaremos crear un nuevo dataset de origen llamado **DS_Onprem_File_param** y un dataset de destino llamado **DS_ADLS_ingest_param** que utilizarán parámetros.


### **Paso 2 - Data Transformation**
_____

1. Pasos en la transformación de datos

    *   Crear una aplicación Azure Synapse Analytics
    *   Crear un Synapse Notebook
    *   Leer los datos que están presentes en Azure Datalake.
    *   Aplicar la lógica de transformación
    *   Escribir los datos en Azure Datalake
```
```
2. Lectura de datos del ADLS desde Synapse Notebook

<center><img src="https://i.postimg.cc/bwn6dD7K/adf519.png"></center>

##### **Paso 2.1 - Creación de Linked Service que haga referencia hacia ADLS**

1. Acceder a ADLS usando **Managed Identity**

2. Configurar la **Managed Identity** de Azure Synapse Analytics y asignarle un **Rol** para que tenga acceso a **ADLS** 

3. Crear el Linked Service **LS_ADLS_transform** que hace referencia a ADLS

##### **Paso 2.2 - Creación de un Synapse notebook y leer los datos de ADLS utilizando Spark Pool**

1. Crear un notebook y conectarse a ADLS usando **PySpark**

##### **Paso 2.3 - Comenzar la lógica de transformación de los datos**

1. Comenzar la lógica de transformación de los datos

    *   Identificar y eliminar filas duplicadas
    *   Reemplazar valores Null
    *   Creación de nuevas columnas
    *   Cambio de tipo de datos
    *   Renombrar columnas

##### **Paso 2.4 - Escribir datos transformados en Datalake**

1. Escribir los datos ya transformados en nuestro ADLS

##### **Paso 2.5 - Llamar al Synapse notebook desde Azure Data Factory**

1. Ambos **Managed Identity** de **ADF** y **User** deben tener acceso **Synapse Administrator** en **Synapse Analytics**

2. Crear un Linked Service que haga referencia a nuestro workspace de Azure Synapse     

3. Uso de la actividad **Notebook** en Azure Data Factory (ADF)  

### **Paso 3 - Data Loading**
_____

1. Acceder a Azure SQL Database desde SSMS

2. Cargar datos hacia Azure SQL Database

<center><img src="https://i.postimg.cc/h4Myztq2/adf564.png"></center>

##### **Paso 3.1 - Acceder a Azure SQL Database desde SSMS**

##### **Paso 3.2 - Creación de Linked Service que haga referencia hacia Azure SQL Database**

1. Enmascarar nuestro Password de acceso al servidor de Azure SQL Database utilizando Azure Key Vault

2. Crear el Linked Service de destino **LS_SQL_Load** que hace referencia a Azure SQL Database. Reutilizaremos el Linked Service de origen **LS_adls_ingest** que hace referencia a nuestro ADLS y también el Linked Service **LS_Keyvault** que hace referencia a Azure Key Vault

##### **Paso 3.3 - Creación de Datasets de origen y destino**

1. Crear el Dataset de origen **DS_ADLS_Refined** que hace referencia a nuestro ADLS, a la ruta **refined/data**. 
    Siendo **refined** el contenedor y **data** el directorio

2. Crear el Dataset de destino **DS_SQL_Load** que hace referencia a Azure SQL Database

##### **Paso 3.3 - Creación de un Pipeline**

1. Crear el Pipeline de ingesta de datos **PL_ADLS_to_SQL**, el cual contendrá una actividad **Copy data**

### **Paso 4 - Mejoras**
_____

1. Mejoras para la copia desde on-premise a ADLS 

    *   Copiar el archivo de hoy a una carpeta específica en ADLS mientras se ingiere desde on-premise.
    *   El nombre de la carpeta debe ser la fecha de hoy (la misma fecha que el archivo).
    *   Todos los archivos de la fecha de hoy se almacenarán en una carpeta que tenga la fecha de hoy.

<center><img src="https://i.postimg.cc/BvqSBfpZ/adf571.png"></center>   

2. Mejoras en el Synapse notebook

    *   La transformación se hará sólo a ese archivo en particular ya que la carpeta tendrá sólo el archivo de hoy
    *   Los recursos de computación se pueden guardar en Spark
    *   El tiempo de ejecución se reducirá ya que sólo se computarán las filas de hoy

<center><img src="https://i.postimg.cc/k4YYhWdY/adf572.png"></center>     

##### **Paso 4.1 - Mejoras para la copia desde on-premise a ADLS**

##### **Paso 4.2 - Mejoras en el Synapse notebook**

### **Paso 5 - Orquestación**
_____

1. Orquestar todos los Pipelines creados

    *   Crear un nuevo Pipeline llamado **PL_Orchestrate_all** y agregar la actividad **Execute Pipeline**
    *   Utilizar un Trigger **Tumbling Window**

<center><img src="https://i.postimg.cc/s2TSnpLG/adf580.png"></center>  

##### **Paso 5.1 - Orquestar todos los Pipelines creados**

##### **Paso 5.2 - Enviar una notificación automática de alerta por correo electrónico cuando falla el pipeline en ADF**

- Como el pipeline está ahora automatizado, necesitamos saber si el pipeline ha fallado
- El correo electrónico de alerta debe enviarse a un grupo o a un usuario individual
- El correo electrónico de alerta debe tener:

    *   El nombre del pipeline
    *   Data Factory name
    *   Run ID


### **Paso 6 - Reporteria (Power BI)**
_____

1. Utilizar los datos que cargamos en Azure SQL Database en un reporte que crearemos en Power BI

    *   Abrir Power BI Desktop
    *   Obtener datos desde Azure SQL Database desde Power BI
    *   Autenticación
    *   Cargar datos en el modelo

##### **Paso 6.1 - Crear un reporte en Power BI**

### **Paso 7 - Configuring Continuous Integration / Continuous deployment**
_____

**Continuous Integration**

<center><img src="https://i.postimg.cc/SQvC35WQ/adf620.png"></center>  

**Continuous Deployment**

<center><img src="https://i.postimg.cc/vDwBMWYh/adf621.png"></center>  
